### HE_Cifar10 Original Pipeline

In [2]:
print("Hello")

Hello


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets
import cv2
import numpy as np
from PIL import Image

# -------- Non-differentiable filter: Histogram Equalization (grayscale) --------
def histogram_equalization(img_tensor):
    img_np = img_tensor.permute(1, 2, 0).numpy() * 255  # [C, H, W] → [H, W, C], [0, 1] → [0, 255]
    img_np = img_np.astype(np.uint8)

    # Convert to grayscale and apply histogram equalization
    gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)
    eq = cv2.equalizeHist(gray)
    
    # Convert back to RGB (grayscale to RGB)
    eq_rgb = cv2.cvtColor(eq, cv2.COLOR_GRAY2RGB)
    eq_tensor = torch.from_numpy(eq_rgb / 255.0).float().permute(2, 0, 1)  # Back to [C, H, W]
    return eq_tensor

# -------- Small CNN --------
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(1)
        )
        self.fc = nn.Linear(32, num_classes)

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

# -------- Dataset Loader (CIFAR10 as example) --------
transform_raw = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor()
])

train_data = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_raw)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)

# -------- Training loop --------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN(num_classes=10).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

for epoch in range(10):  # Train only one epoch for testing
    for images, labels in train_loader:
        # Apply the non-differentiable filter to each image
        filtered = torch.stack([histogram_equalization(img) for img in images])
        filtered = filtered.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(filtered)
        loss = criterion(outputs, labels)

        # Backward + Optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch finished. Loss: {loss.item():.4f}")
# -------- Load test dataset --------
test_data = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_raw)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)


# -------- Evaluation function --------
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            # Apply non-differentiable filter
            filtered = torch.stack([histogram_equalization(img) for img in images])
            filtered = filtered.to(device)
            labels = labels.to(device)

            outputs = model(filtered)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f"Test Accuracy: {accuracy * 100:.2f}%")
    model.train()
    return accuracy

# -------- Evaluate the trained model --------
evaluate_model(model, test_loader)

# -------- Save the model --------
torch.save(model.state_dict(), "camera_pipeline_model.pth")
print("Model saved to camera_pipeline_model.pth")


Epoch finished. Loss: 1.7871
Epoch finished. Loss: 2.0357
Epoch finished. Loss: 2.0957
Epoch finished. Loss: 1.9613
Epoch finished. Loss: 1.5575
Epoch finished. Loss: 1.5815
Epoch finished. Loss: 1.6980
Epoch finished. Loss: 1.5047
Epoch finished. Loss: 1.9394
Epoch finished. Loss: 1.6374
Test Accuracy: 41.78%
Model saved to camera_pipeline_model.pth


### Neural Surrogate

In [10]:
class DifferentiableHE(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 3, kernel_size=3, padding=1),
            nn.Sigmoid()  # Keep output in [0, 1]
        )

    def forward(self, x):
        return self.net(x)


In [11]:
diff_he_model = DifferentiableHE().to(device)
he_optimizer = torch.optim.Adam(diff_he_model.parameters(), lr=1e-3)
he_criterion = nn.L1Loss()  # or nn.MSELoss()

# Pretrain for 1 epoch
for images, _ in train_loader:
    images = images.to(device)
    with torch.no_grad():
        target_he = torch.stack([histogram_equalization(img.cpu()) for img in images]).to(device)

    output = diff_he_model(images)
    loss = he_criterion(output, target_he)

    he_optimizer.zero_grad()
    loss.backward()
    he_optimizer.step()

print(f"[Pretraining HE Approx] Loss: {loss.item():.4f}")


[Pretraining HE Approx] Loss: 0.1327


In [12]:
torch.save(diff_he_model.state_dict(), "pretrained_he.pth")

In [6]:
# Forward through differentiable approximation
filtered = diff_he_model(images.to(device))
outputs = model(filtered)

In [7]:
class FullCameraPipeline(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.he = DifferentiableHE()
        self.cnn = SimpleCNN(num_classes=num_classes)

    def forward(self, x):
        x = self.he(x)
        return self.cnn(x)


In [15]:
# 1. Initialize full model
model = FullCameraPipeline(num_classes=10).to(device)

# 2. Load pretrained HE weights
pretrained_he = torch.load("pretrained_he.pth")
model.he.load_state_dict(pretrained_he)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

for epoch in range(10):  # More epochs now
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/5] - Loss: {avg_loss:.4f}")


Epoch [1/5] - Loss: 1.9787
Epoch [2/5] - Loss: 1.7711
Epoch [3/5] - Loss: 1.7073
Epoch [4/5] - Loss: 1.6603
Epoch [5/5] - Loss: 1.6043
Epoch [6/5] - Loss: 1.5522
Epoch [7/5] - Loss: 1.5073
Epoch [8/5] - Loss: 1.4642
Epoch [9/5] - Loss: 1.4349
Epoch [10/5] - Loss: 1.4078


In [17]:
evaluate_model(model, test_loader)

Test Accuracy: 28.77%


0.2877